In [ ]:
# Import necessary libraries and your custom factory
import os
import pandas as pd
from src.Ingest_Data import DataIngestorFactory


#-------#
# To help with reading and manipulating data
import pandas as pd
import numpy as np

# To help with data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# To be used for missing value imputation
from sklearn.impute import SimpleImputer

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To get different metric scores, and split data
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score
)

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To be used for tuning the model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# To use statistical functions
import scipy.stats as stats

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
import warnings

warnings.filterwarnings("ignore")

In [ ]:
#Load Data
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"

In [ ]:
def load_data(df) -> pd.DataFrame:
    try:
        df = DataIngestorFactory.get_data_ingestor(TRAIN_ENDPOINT).ingest(TRAIN_ENDPOINT)
        print("\n✅ Successfully ingested test data from the API.")
        display(df.head())
        print(f"\nShape of test data: {df.shape}")
    except ConnectionError as e:
        print(f"❌ CONNECTION FAILED: Could not connect to the API.")
        print(f"Please ensure your FastAPI server is running on {BASE_URL}")
        print(f"Error details: {e}")
    return df

In [ ]:
df_train = load_data(TRAIN_ENDPOINT)
df_test = load_data(TEST_ENDPOINT)

## Data Overview

In [ ]:
df_train.tail()

In [ ]:
df_train.info()

In [ ]:
# let's check for duplicate values in the data
df_train.duplicated().sum()

In [ ]:
df_train.nunique()

## Univariate Analysis

In [ ]:
cols = df_train.columns.tolist()

plt.figure(figsize=(20, 50))
for i, variable in enumerate(cols):
    plt.subplot(14, 3, i + 1)
    sns.boxplot(df_train[variable])
    plt.tight_layout()
    plt.title(variable)
plt.show()

: 

: 

Error in callback <function _draw_all_if_interactive at 0x000001BC11471260> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(df_train["Target"])
plt.show()
df_train["Target"].value_counts()

- "Target" class is imbalanced with 37813 or 94.53% "No failures (i.e., 0)" and 2187 or 5.47% "Failures (i.e., 1)"

## Data Pre-processing